## Loading and preparing JSON data

We load jsons an save for each classified window. We'll save a big file with subject 
name, frame; And later we'll add subject class and frame class. 
Where subject class is overall normal or abnormal, and frame class is tagged as 
if used for study (GM).

In [10]:
import json
import os
import pandas as pd


data = []
data_folder = '/home/harrisonford/Documents/babybrain/confidences/'
data_files = sorted(os.listdir(data_folder))
for a_file in data_files:
    with open(data_folder + a_file) as json_file:
        single_data = json.load(json_file)
        my_frames = single_data['processed_frames']
        my_confidences = single_data['confidences']
        for frame_value, confidence_value in zip(my_frames, my_confidences):
            data.append([a_file[:-5], frame_value, confidence_value])

# create a DataFrame
df = pd.DataFrame(data, columns=['subject', 'frame', 'confidences'])

# save it as csv
df.to_csv('confidences.csv')

## Loading label data

We load the excel files with specialist classification, we'll add to prior data two 
columns containing subject class (1=normal, 0=abnormal) and 

In [ ]:
label_path = '/home/harrisonford/Documents/babybrain/SubjectTags.xlsx'